## Estadística Aplicada

Actividad 6     
26 de septiembre

Francisco Javier Chávez Hernández 2034204

Gpo 044

In [44]:
#librerias basicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [45]:
#librerias especializadas
from lifelines import CoxPHFitter
import seaborn as sns

In [46]:
#importar base de datos
datos=pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv')
datos.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,6.128335,7.161017,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,2.486307,1.711691,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,4.400000,5.600000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,6.400000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,8.200000,8.700000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [47]:
datos.head()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo


## 3) depurar la base de datos

In [48]:
#a)Verificar que no haya missing values, en caso de haberlos decidir imputar o eliminar
datos.isnull().sum()

satisfaccion           0
evaluacion             0
proyectos              0
promedio_horas_mes     0
semestre               0
accidente              0
renunciar              0
ascenso_5years         0
departamento          26
salario                0
dtype: int64

In [49]:
datos.count()

satisfaccion          14999
evaluacion            14999
proyectos             14999
promedio_horas_mes    14999
semestre              14999
accidente             14999
renunciar             14999
ascenso_5years        14999
departamento          14973
salario               14999
dtype: int64

decidí elminar porque los datos perdidos es en una variable categórica y no numérica a la que podría imputar con la media/promedio de las demás observasiones

In [50]:
datos=datos.dropna()

In [51]:
datos.count()

satisfaccion          14973
evaluacion            14973
proyectos             14973
promedio_horas_mes    14973
semestre              14973
accidente             14973
renunciar             14973
ascenso_5years        14973
departamento          14973
salario               14973
dtype: int64

In [52]:
#b)Verificar que no haya valores duplicados con dataframe.duplicated().sum() en caso de haber valores duplicados eliminarlos con 
#dataframe=dataframe.drop_duplicates(keep='first').reset_index(drop=True)

datos.duplicated().sum()

2982

In [53]:
datos=datos.drop_duplicates(keep='first').reset_index(drop=True)
datos.duplicated().sum()

0

In [54]:
datos.count()

satisfaccion          11991
evaluacion            11991
proyectos             11991
promedio_horas_mes    11991
semestre              11991
accidente             11991
renunciar             11991
ascenso_5years        11991
departamento          11991
salario               11991
dtype: int64

In [55]:
#c) Verificar que todas las variables sean numéricas, si no son numéricas hacerlas numéricas, en el caso de las categóricas convertirlas en 
#dummies y conservar n-1 dummies
datos.dtypes

satisfaccion          float64
evaluacion            float64
proyectos               int64
promedio_horas_mes      int64
semestre                int64
accidente               int64
renunciar               int64
ascenso_5years          int64
departamento           object
salario                object
dtype: object

In [56]:
#ventas fuera
dummies1 = pd.get_dummies(datos['departamento']).astype(int)
dummies1 = dummies1[['Investigación', 'administración', 'contabilidad', 'desarrollo de producto', 'informática', 'mantenimiento', 'mercadotecnia', 'recursos humanos', 'soporte']]


In [57]:
#salario bajo fuera para poder comparar mejor
dummies2 = pd.get_dummies(datos['salario']).astype(int)
dummies2 = dummies2[['alto', 'medio']]

In [58]:
datos = pd.concat([datos,dummies1],axis=1)
datos = datos.drop('departamento',axis=1)

datos = pd.concat([datos,dummies2],axis=1)
datos = datos.drop('salario',axis=1)


In [59]:
datos.head()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,Investigación,administración,contabilidad,desarrollo de producto,informática,mantenimiento,mercadotecnia,recursos humanos,soporte,alto,medio
0,3.8,5.3,2,157,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,8.0,8.6,5,262,6,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,1.1,8.8,7,272,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,7.2,8.7,5,223,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,3.7,5.2,2,159,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
#d) Calcular estadísticas descriptivas
datos.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,Investigación,administración,contabilidad,desarrollo de producto,informática,mantenimiento,mercadotecnia,recursos humanos,soporte,alto,medio
count,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000
mean,6.296581,7.166825,3.802852,200.473522,3.364857,0.154282,0.166041,0.016929,0.057877,0.036361,0.051789,0.057210,0.081394,0.18714,0.056125,0.050121,0.151864,0.082562,0.438746
std,2.410700,1.683426,1.163238,48.727813,1.330240,0.361234,0.372133,0.129012,0.233520,0.187194,0.221610,0.232252,0.273451,0.39004,0.230173,0.218204,0.358904,0.275230,0.496254
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.800000,5.700000,3.000000,157.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.600000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.200000,8.600000,5.000000,243.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


In [61]:
1-.438746-.082562

0.4786919999999999

Ya con los datos "pulidos" se tiene que el promedio de renuncias fue de 16.6%, se trabajaron en promedio 200 horas por mes por empleado (muy alta en mi opinión). Aunado a esto se tiene que el 47.87% de los empleados tiene un salario bajo lo cual es un dato preocupante, esto podría ser un indicador del alto indice de renuncias en la empresa. Y la mayor concentración de empleados está en el área de mantenimiento con 18.71%, mientras la menor en administración con 3.63%

## 4) Estimar el modelo de Cox e interpretar cada uno de los coeficientes

In [64]:
cphf=CoxPHFitter().fit(datos,'semestre','renunciar')
cphf.print_summary()

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
      baseline estimation = breslow
   number of observations = 11991
number of events observed = 1991
   partial log-likelihood = -15884.05
         time fit was run = 2023-09-27 02:10:24 UTC

---
                         coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                      
satisfaccion            -0.24       0.78       0.01            -0.26            -0.22                 0.77                 0.80
evaluacion              -0.01       0.99       0.01            -0.04             0.02                 0.96                 1.02
proyectos               -0.29       0.75       0.02            -0.33            -0.25                 0.72                 0.78
promedio_horas_mes       0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
accidente               -1.25       0.29       0.10            -1.44            -1.05                 0.24                 0.35
ascenso_5years          -1.38       0.25       0.36            -2.08            -0.68                 0.13                 0.51
Investigación           -0.35       0.70       0.12            -0.58            -0.12                 0.56                 0.89
administración          -0.13       0.88       0.15            -0.42             0.16                 0.66                 1.17
contabilidad            -0.11       0.90       0.10            -0.32             0.10                 0.73                 1.10
desarrollo de producto   0.01       1.01       0.10            -0.20             0.21                 0.82                 1.24
informática             -0.06       0.94       0.09            -0.24             0.12                 0.79                 1.13
mantenimiento            0.08       1.08       0.07            -0.05             0.21                 0.95                 1.24
mercadotecnia           -0.01       0.99       0.10            -0.22             0.19                 0.81                 1.21
recursos humanos         0.08       1.09       0.10            -0.12             0.29                 0.89                 1.33
soporte                  0.10       1.11       0.07            -0.04             0.24                 0.96                 1.27
alto                    -1.54       0.21       0.15            -1.83            -1.25                 0.16                 0.29
medio                   -0.39       0.68       0.05            -0.48            -0.30                 0.62                 0.74

                         cmp to      z      p   -log2(p)
covariate                                               
satisfaccion               0.00 -26.89 <0.005     526.58
evaluacion                 0.00  -0.76   0.44       1.17
proyectos                  0.00 -14.75 <0.005     161.15
promedio_horas_mes         0.00   3.88 <0.005      13.25
accidente                  0.00 -12.41 <0.005     115.04
ascenso_5years             0.00  -3.88 <0.005      13.21
Investigación              0.00  -3.01 <0.005       8.56
administración             0.00  -0.89   0.37       1.42
contabilidad               0.00  -1.05   0.29       1.77
desarrollo de producto     0.00   0.09   0.93       0.11
informática                0.00  -0.65   0.52       0.95
mantenimiento              0.00   1.23   0.22       2.18
mercadotecnia              0.00  -0.11   0.91       0.14
recursos humanos           0.00   0.81   0.42       1.26
soporte                    0.00   1.42   0.16       2.67
alto                       0.00 -10.36 <0.005      81.15
medio                      0.00  -8.39 <0.005      54.18
---
Concordance = 0.85
Partial AIC = 31802.11
log-likelihood ratio test = 1360.94 on 17 df
-log2(p) of ll-rati

In [65]:
cphf.params_

covariate
satisfaccion             -0.242337
evaluacion               -0.010687
proyectos                -0.291606
promedio_horas_mes        0.001839
accidente                -1.246686
ascenso_5years           -1.380084
Investigación            -0.350547
administración           -0.130262
contabilidad             -0.110158
desarrollo de producto    0.009468
informática              -0.058469
mantenimiento             0.081424
mercadotecnia            -0.011698
recursos humanos          0.083791
soporte                   0.100538
alto                     -1.538901
medio                    -0.391289
Name: coef, dtype: float64

In [66]:
cphf.hazard_ratios_

covariate
satisfaccion              0.784792
evaluacion                0.989370
proyectos                 0.747063
promedio_horas_mes        1.001841
accidente                 0.287456
ascenso_5years            0.251557
Investigación             0.704303
administración            0.877865
contabilidad              0.895692
desarrollo de producto    1.009513
informática               0.943208
mantenimiento             1.084830
mercadotecnia             0.988370
recursos humanos          1.087401
soporte                   1.105766
alto                      0.214617
medio                     0.676185
Name: exp(coef), dtype: float64

**satisfacción** p_valor < 0.05. Se rechaza $H_0$, la satisfaccion del empleado influye en el riesgo de renuncia del empleado y mientras mayor sea su satisfacción disminuye el riesgo de renunciar en 22%.

**evaluación** p_valor > 0.05. No se rechaza $H_0$, la evaluación no influye en el riesgo de renuncia del empleado.

**proyectos** Se rechaza $H_0$, el número de proyectos influye en el riesgo de renuncia del empleado. Si la cantidad de proyectos es mayor, el riesgo de renuncia del empleado disminuye en 25% por cada proyecto.

**promedio_horas_mes** Se rechaza $H_0$, el promedio de horas al mes influye en el riesgo de renuncia, mientras mas horas promedio al mes aumenta el riesgo de renuncia del empleado en un 0.1841% por cada hora en promedio al mes.

**accidentes** Se rechaza $H_0$, si un empleado sufre un accidente en la empresa influye en el riesgo de su renuncia y lo disminuye en 71.26% el riesgo de renunciar del empleado en conmparacoin con los que no se han accidentado en la empresa.

**ascenso_5years** Se rechaza $H_0$, un ascenso en los ultimos 5 años influye en el riesgo de renuncia del empleado disminuyendo su riesgo en 74.85% en comparacion con los que no son ascendidos en los ultimos 5 años.

**departamento de Investigacion** Se rechaz $H_0$, que los empleados pertenezcan al departamento de investigacion influye en el riesgo de renunciar disminuyendo el riesgo en un 29.57% en comparacoin con el departamento de ventas(Dummie que se eliminó).

**departamentos de administracion, contabilidad, desarrollo de productos, informatica, mantenimiento, mercadotecnia, recursos humanos y soporte** En todos su p_valor > 0.05, por lo que no se rechaza $H_0$, entonces que los empleados pertenezcan a estos departamentos no influye en el riesgo de renuncia en comparacion con el departamento de ventas.

**salario alto** Se rechaza $H_0$, que el empleado tenga un salario alto influye en el riesgo de renuncia del empleado, disminuyéndolo en 78.54% en comparación con los que tienen un salario bajo. 

**salario medio** Se rechaza $H_0$, que el empleado tenga un salario medio influye en el riesgo de renuncia del empleado, disminuyendolo en 32.39% en comparación con los que tienen un salario bajo. 

## 5) Con base en la interpretación de los coeficientes, ¿qué recomendación podría hacerle a la empresa para que disminuya la cantidad de renuncias de los empleados?

Con base a los datos, lo que ayudaría a disminuir la cantidad de renuncias de empleados es:

1. Aumentar los salarios, la brecha salarial es considerable
2. Ascender con mayor frecuencia, obviamente considerando desempeño. En lugar de tomar 5 años como referencia una marca menor (3 años) mejoraría este índice
3. Sufrir un accidente en la empresa disminuye en gran cantidad las renuncias, pero obviamente disminuir los accidentes mejora la calidad de trabajo y vida de los empleados, además de que un accidentado no puede trabajar, así que este se descarta por lógica


Opinión extra:
El promedio de horas debería tener un porcentaje de riesgo mayor, México es el país con mayor horas semanales de trabajo y se normalizó de manera que el empleado no recibe lo que debería merecer. En Estados Unidos se trabaja en promedio 40horas por semana, es decir 170 por mes aproximadamente

## 6) Pruebe si el modelo de Cox cumple el supuesto de riesgos proporcionales, de no hacerlo estime el modelo AFT adecuado

In [68]:
cphf.check_assumptions(datos,p_value_threshold=0.05)

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
         test_name = proportional_hazard_test

---
                             test_statistic      p  -log2(p)
Investigación          km              1.04   0.31      1.70
                       rank            4.36   0.04      4.76
accidente              km              0.03   0.87      0.21
                       rank            1.14   0.29      1.81
administración         km              3.66   0.06      4.17
                       rank            1.53   0.22      2.21
alto                   km              0.37   0.54      0.88
                       rank            0.00   0.96      0.06
ascenso_5years         km              3.66   0.06      4.17
                       rank            6.71   0.01      6.71
contabilidad           km              0.52   0.47      1.09
                       rank            0.71   0.40      1.33
desarrollo de producto km              0.07   0.79      0.34
                       rank            1.10   0.29      1.77
evaluacion             km            443.05 <0.005    324.32
                       rank          427.15 <0.005    312.82
informática            km              0.08   0.77      0.37
                       rank            2.59   0.11      3.22
mantenimiento          km              0.31   0.58      0.79
                       rank            0.91   0.34      1.56
medio                  km              0.44   0.51      0.98
                       rank            0.75   0.39      1.37
mercadotecnia          km              1.24   0.27      1.91
                       rank            1.31   0.25      1.99
promedio_horas_mes     km            270.84 <0.005    199.74
                       rank          307.41 <0.005    226.21
proyectos              km            574.13 <0.005    419.05
                       rank          684.93 <0.005    499.11
recursos humanos       km              0.00   0.99      0.01
                       rank            0.00   1.00      0.00
satisfaccion           km           1257.74 <0.005    912.74
                       rank          845.24 <0.005    614.90
soporte                km              2.68   0.10      3.30
                       rank            3.25   0.07      3.81



1. Variable 'satisfaccion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'satisfaccion' might be incorrect. That is, there
may be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'satisfaccion' using pd.cut, and then specify it in
`strata=['satisfaccion', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'evaluacion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'evaluacion' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to 

[]

## NO CUMPLE CON LOS SUPUESTOS

In [71]:
from lifelines.fitters.log_normal_fitter import LogNormalFitter
from lifelines.fitters.weibull_fitter import WeibullFitter
from lifelines import KaplanMeierFitter, ExponentialFitter, WeibullAFTFitter, LogNormalAFTFitter

In [73]:
mExp = ExponentialFitter().fit(datos['semestre'],datos['renunciar'])
mWeibull = WeibullFitter().fit(datos['semestre'],datos['renunciar'])
mLogN = LogNormalFitter().fit(datos['semestre'],datos['renunciar'])

mExp.AIC_, mWeibull.AIC_, mLogN.AIC_

(15965.45892201228, 12511.512867661068, 11587.92169210356)

El LogNormal es el más apto por tener el AIC menor

In [76]:
modelo=LogNormalAFTFitter().fit(datos, 'semestre', 'renunciar')
modelo.print_summary(decimals=4)

<lifelines.LogNormalAFTFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
   number of observations = 11991
number of events observed = 1991
           log-likelihood = -4876.5727
         time fit was run = 2023-09-27 04:36:36 UTC

---
                                 coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                                       
mu_    Investigación           0.0574     1.0590     0.0269           0.0046           0.1101               1.0046               1.1164
       accidente               0.2630     1.3009     0.0203           0.2233           0.3028               1.2502               1.3536
       administración          0.1053     1.1111     0.0327           0.0412           0.1695               1.0421               1.1847
       alto                    0.3700     1.4477     0.0296           0.3119           0.4280               1.3660               1.5342
       ascenso_5years          0.3650     1.4406     0.0629           0.2417           0.4883               1.2735               1.6296
       contabilidad            0.0141     1.0142     0.0254          -0.0356           0.0638               0.9650               1.0659
       desarrollo de producto  0.0055     1.0056     0.0250          -0.0434           0.0545               0.9575               1.0560
       evaluacion              0.0057     1.0057     0.0039          -0.0019           0.0134               0.9981               1.0135
       informática             0.0110     1.0110     0.0217          -0.0315           0.0534               0.9690               1.0549
       mantenimiento          -0.0203     0.9799     0.0162          -0.0520           0.0114               0.9494               1.0115
       medio                   0.0839     1.0875     0.0112           0.0620           0.1058               1.0639               1.1116
       mercadotecnia           0.0218     1.0220     0.0252          -0.0276           0.0712               0.9728               1.0738
       promedio_horas_mes     -0.0003     0.9997     0.0001          -0.0006          -0.0000               0.9994               1.0000
       proyectos               0.0961     1.1009     0.0052           0.0859           0.1063               1.0897               1.1122
       recursos humanos       -0.0289     0.9716     0.0253          -0.0785           0.0207               0.9245               1.0209
       satisfaccion            0.0670     1.0693     0.0023           0.0625           0.0716               1.0645               1.0742
       soporte                -0.0257     0.9747     0.0173          -0.0595           0.0082               0.9422               1.0082
       Intercept               0.9144     2.4954     0.0274           0.8607           0.9681               2.3649               2.6330
sigma_ Intercept              -1.1138     0.3283     0.0152          -1.1435          -1.0840               0.3187               0.3382

                                cmp to        z      p   -log2(p)
param  covariate                                                 
mu_    Investigación            0.0000   2.1322 0.0330     4.9218
       accidente                0.0000  12.9813 <5e-05   125.5891
       administración           0.0000   3.2194 0.0013     9.6043
       alto                     0.0000  12.4914 <5e-05   116.5323
       ascenso_5years           0.0000   5.8026 <5e-05    27.1903
       contabilidad             0.0000   0.5550 0.5789     0.7886
       desarrollo de producto   0.0000   0.2222 0.8241     0.2790
       evaluacion               0.0000   1.4625 0.1436     2.7999
       informática              0.0000   0.5061 0.6128     0.7066
       mantenimiento            0.0000  -1.2545 0.2096     2.2540
     

## 7. Interprete los coeficientes del modelo AFT

**Investigacion:** p_valor < 0.05. Se rechaza $H_0$, ser empleado del area de investigacion afecta al riesgo de renunciar del empleado, aumenta el riesgo en 5.9% en comparacion con los empleados del area de ventas.

**accidente:**  Se rechaza $H_0$, accidentarse en la empresa afecta al riesgo de renunciar del empleado aumentando el riesgo en 30.1% en comparación con los empleados que no se accidentan.

**administración:** Se rechaza $H_0$, ser empleado del area de administracion afecta en el riego de renunciar del empleado y este riesgo aumenta en 11% en comparacion con los empleados del area de ventas.

**salario alto:**  Se rechaza $H_0$, el tener un sueldo con clasificacion alto afecta en el riesgo de renunciar del empleado y lo aumenta en 45% en comparacoin con los que tienen sueldo con clasificacion bajo

**ascenso_5years:** Se rechaza $H_0$, ser ascendido en los ultimos 5 años aumenta el riesgo de renunciar en 44.1% en comparación con los que no han sido ascendidos en los ultimos 5 años.

**contabilidad, desarrollo de producto, informática, mantenimiento, mercadotecnia, recursos humanos, soporte:** Todos los p-valores<0.05, por lo que no se rechaza $H_0$, ser empleado de cualquiera de estas áreas no aumenta el riesgo de renunciar en comparación con ser del área de ventas.

**evaluacion:** No se rechaza $H_0$, la evaluación del empleado no afecta el riesgo de renunciar.

**Sueldo-medio:**  Se rechaza $H_0$, tener un sueldo medio afecta en el riesgo de renunciar aumentandolo en 8.7% en comparación de tener un sueldo bajo.

**promedio_horas_mes**  Se rechaza $H_0$, el promedio de horas mensuales trabajadas influyen en el riesgo de renunciar disminuyendolo en 0.03% por cada hora en promedio al mes. 

**proyectos** Se rechaza $H_0$, el número de proyectos influye en el riesgo de renunciar aumentándolo en 10.09% por cada proyecto asignado al empleado.

**satisfaccion** Se rechaza $H_0$, la satisfacción del empleado influye en el riesgo de renunciar aumentandolo en 6.93% por cada nivel en la escala de satisfacción del empleado.